In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
df = pd.read_csv("/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv")

In [ ]:
df.head()

In [ ]:
df.shape

Converting the first DICOM image into an array of values:

In [ ]:
import pydicom
import seaborn as sns

In [ ]:
image = pydicom.dcmread('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/9a5094b2563a1ef3ff50dc5c7ff71345.dicom')
image

In [ ]:
image.PatientAge # Access patient metadata from the dcmread function's output

In [ ]:
df.iloc[2]['x_min']

Adding a bounding box on above image, based on x, y coordinates in the train.csv file

In [ ]:
fig, ax = plt.subplots(1)
ax.imshow(image.pixel_array, cmap = "gray")
length = df.iloc[2]['x_max'] - df.iloc[2]['x_min']
width = df.iloc[2]['y_max'] -  df.iloc[2]['y_min']
box = matplotlib.patches.Rectangle((df.iloc[2]['x_min'], df.iloc[2]['y_min']), length, width, linewidth=1, facecolor='none', edgecolor = 'r')
ax.add_patch(box)
plt.show()

EDA of the classes present and understanding the distribution of classes

In [ ]:
# Spread of images across classes
sns.distplot(df['class_id'], kde = False)

This dataset is unbalanced and needs to be balanced for predictions.

In [ ]:
len(df[df['class_id'] == 14])   # Number of rows corresponding to "Not identified"

In [ ]:
67914 - 31818 # Total number of records that have a specific category (total - unidentified)

In [ ]:
len(df.dropna(subset=["x_min", "x_max", "y_min", "y_max"]))
# Once all Nan values for x_min, x_max, y_min, y_max are removed, we get the same number as above
# Conclusion: If x, y coordinates are not known, then that row belongs to type 14
# This can be checked by removing all Nan rows and checking if any are of type 14 in the pending df

In [ ]:
ided_df = df.dropna(subset=["x_min", "x_max", "y_min", "y_max"])

In [ ]:
ided_df['class_id'].unique() # Check unique values for all remaining objects, after removing Nan
# Here class_id 14 is not present, proving above hypothesis.
# Thus, if x and y coordinates are missing, it cannot be identified as belonging to any class
# The model needs to be trained only on the remaining set of values

In [ ]:
ided_df.head()

In [ ]:
# Plotting distribution of classes in the new df
sns.distplot(ided_df["class_id"], kde = False)

Classes 0, 3, 11 and 13 still have a large number of samples as compared to other classes.